## Deployment

Deployment notes and scripts

Prior to starting Jupyter be sure node js is avaiable - set it via nvm if needed.

NOTE: This copy of the book deploys two domains to the same test net. This is done to troubleshoot the set up prior to two chain deployment.

In [2]:
# Eth test account addresses
ACCT0='0x892BB2e4F6b14a2B5b82Ba8d33E5925D42D4431F'
ACCT1='0x9949f7e672a568bB3EBEB777D5e8D1c1107e96E5'
ACCT2='0x835F0Aa692b8eBCdEa8E64742e5Bce30303669c2'
ACCT3='0x7bA7d161F9E8B707694f434d65c218a1F0853B1C'
ACCT4='0xB4C3D79CDC0eb7A8576a8bf224Bbc6Bec790c320'
ACCT5='0x5Ad35F89D8C1d03089BDe2578Ce43883E3f2A7B0'
ACCT6='0x0234643975F308b76d1241897e7d70b02C155daa'
ACCT7='0x5199524B11e801c52161CA76dB9BFD72f4a4E1E1'
ACCT8='0x549381D65fe61046911d11743D5c0941Ed704640'
ACCT9='0x73dA1eD554De26C467d97ADE090af6d52851745E'

In [3]:
# Eth test account private keys
ACCT0KEY='0xcb1a18dff8cfcee16202bf86f1f89f8b3881107b8192cd06836fda9dbc0fde1b'
ACCT1KEY='0xa54f24f80839b659fc44fbb19492507bc734ea572f6f5672787cd8e9a198bf28'
ACCT2KEY='0x824f9e081f93102ca26e9e696bb6804079a5e4e3fca3a05216e2b6e0538fcab9'
ACCT3KEY='0x3e56e9b2db8389123a03816c37dd4515e07077483fbe865156a2d8f6003b6725'
ACCT4KEY='0xca55c5904f97405816f8d24c5e7810aaec2aa347ea0e1f920f19eca05669ae7b'
ACCT5KEY='0x6100006a16d6a0fd065f62165e64d412920afd0e8fc59956ad7a9116e363b72e'
ACCT6KEY='0x55b0e7919eb08e618dbddf025341094eefc0db60244736c1faac1412d68868f3'
ACCT7KEY='0x81b475051c4686cffe815e50216a9bac397c0e5d9108170823b290d41823fa8a'
ACCT8KEY='0xc3599ced1484dc8eccbb477cf8c318ca48ef1f2aac81a10a3774fe40d3b678de'
ACCT9KEY='0xf9832eeac47db42efeb2eca01e6479bfde00fda8fdd0624d45efd0e4b9ddcd3b'

In [4]:
domain_1_acct = ACCT0
domain_1_key = ACCT0KEY

domain_2_acct = ACCT9
domain_2_key = ACCT9KEY

domain_2_mint_recipient = ACCT4
domain_2_mint_recipient_key = ACCT4KEY

attestor = ACCT8
attestor_key = ACCT8KEY

In [5]:
print(domain_1_acct)

0x892BB2e4F6b14a2B5b82Ba8d33E5925D42D4431F


## Fiddy Deployment - Domain 1

1. Pick account for deploying FIDDY
2. Deploy it
3. Grab the address









In [6]:
home = ! echo $HOME
exec_home = home[0] + '/code/cctp-components'
print(exec_home)

/home/dsmith/code/cctp-components


In [7]:
# Fiddy deployment
!cd {exec_home} && DEPLOYER_KEY={domain_1_key} OWNER_ADDRESS={domain_1_acct} forge script script/Token.s.sol:DeployScript --broadcast --rpc-url http://127.0.0.1:8545 --extra-output-files abi --extra-output-files bin


[⠒] Compiling...
No files changed, compilation skipped
Script ran successfully.

EIP-3855 is not supported in one or more of the RPCs used.
Unsupported Chain IDs: 31337.
Contracts deployed with a Solidity version equal or higher than 0.8.20 might not work properly.
For more information, please see https://eips.ethereum.org/EIPS/eip-3855

## Setting up (1) EVMs.


Chain 31337

Estimated gas price: 3.000005734 gwei

Estimated total gas used for script: 1308258

Estimated amount required: 0.003924781501551372 ETH


###
Finding wallets for all the necessary addresses...
##
Sending transactions [0 - 0].
⠁ [00:00:00] [#################################################] 1/1 txes (0.0s)
Transactions saved to: /home/dsmith/code/cctp-components/broadcast/Token.s.sol/31337/run-latest.json

Sensitive values saved to: /home/dsmith/code/cctp-components/cache/Token.s.sol/31337/run-latest.json

##
Waiting for receipts.
⠉ [00:00:00] [#############################################] 1/1 receipts (0.0s)
###

In [8]:
%%bash -s "$exec_home" --out DOMAIN1_FIDDY_ADDRESS
cd $1
ls
node script/deploy-details.js

In [9]:
DOMAIN1_FIDDY_ADDRESS = DOMAIN1_FIDDY_ADDRESS.strip()
print(DOMAIN1_FIDDY_ADDRESS)

0x13B2DC341d5Ce2F9FF32c7e8aeFAe96ff2A35998


## Fiddy Domain 2 Deployment

1. Pick account for deploying FIDDY
2. Deploy it
3. Grab the address

In [10]:
# PG deployment
!cd {exec_home} && DEPLOYER_KEY={domain_2_key} OWNER_ADDRESS={domain_2_acct} forge script script/Token.s.sol:DeployScript --broadcast --rpc-url http://127.0.0.1:8545 --extra-output-files abi --extra-output-files bin


[⠃] Compiling...
No files changed, compilation skipped
Script ran successfully.

EIP-3855 is not supported in one or more of the RPCs used.
Unsupported Chain IDs: 31337.
Contracts deployed with a Solidity version equal or higher than 0.8.20 might not work properly.
For more information, please see https://eips.ethereum.org/EIPS/eip-3855

## Setting up (1) EVMs.


Chain 31337

Estimated gas price: 3.000005024 gwei

Estimated total gas used for script: 1308258

Estimated amount required: 0.003924780572688192 ETH


###
Finding wallets for all the necessary addresses...
##
Sending transactions [0 - 0].
⠁ [00:00:00] [#################################################] 1/1 txes (0.0s)
Transactions saved to: /home/dsmith/code/cctp-components/broadcast/Token.s.sol/31337/run-latest.json

Sensitive values saved to: /home/dsmith/code/cctp-components/cache/Token.s.sol/31337/run-latest.json

##
Waiting for receipts.
⠉ [00:00:00] [#############################################] 1/1 receipts (0.0s)
###

In [11]:
%%bash -s "$exec_home" --out DOMAIN2_FIDDY_ADDRESS
cd $1
ls
node script/deploy-details.js

In [12]:
DOMAIN2_FIDDY_ADDRESS = DOMAIN2_FIDDY_ADDRESS.strip()
print(DOMAIN2_FIDDY_ADDRESS)

0xdb821F1762df70db9A6E319cE93359Cb10b865d3


## CCTP Set Up - Domain 1


In [13]:
# Adapted from https://github.com/circlefin/evm-cctp-contracts/blob/master/scripts/precomputeRemoteMessengerAddress.py

from web3 import Web3

import os
import rlp

def compute_address(sender, nonce):
    """
    Computes the address for a deployed contract given the address
    of the sender and the current nonce.
    """
    sender_as_bytes = Web3.toBytes(hexstr=sender)
    contract_address = Web3.toHex(Web3.keccak(rlp.encode([sender_as_bytes, nonce])))[-40:]
    return contract_address

def precompute_remote_token_messenger_address(remote_rpc_url, remote_deployer):
    """
    Computes expected address for remote token messenger contract on the
    input remote_rpc_url
    """
    remote_token_messenger_deployer = remote_deployer
    remote_domain_node = Web3(Web3.HTTPProvider(remote_rpc_url))

    remote_token_messenger_deployer_nonce =  remote_domain_node.eth.get_transaction_count(remote_token_messenger_deployer)
    
    remote_token_messenger_address = compute_address(remote_token_messenger_deployer, remote_token_messenger_deployer_nonce)

    return remote_token_messenger_address



/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.11) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [14]:
domain2TokenMessengerAddress = precompute_remote_token_messenger_address('http://localhost:8545', domain_2_acct)
print(domain2TokenMessengerAddress)

71ed4a559c87f6d650a00744b5ccd9687153029a


In [15]:
home = ! echo $HOME
cctp_home = home[0] + '/code/evm-cctp-contracts'
print(cctp_home)

/home/dsmith/code/evm-cctp-contracts


In [16]:
%%bash -s "$cctp_home" "$domain_1_acct" "$domain_1_key" "$DOMAIN1_FIDDY_ADDRESS" "$DOMAIN2_FIDDY_ADDRESS" "$domain2TokenMessengerAddress" "$attestor"

cd $1
ls $1
echo $2
export MESSAGE_TRANSMITTER_DEPLOYER_KEY=$3
export TOKEN_MESSENGER_DEPLOYER_KEY=$3
export TOKEN_MINTER_DEPLOYER_KEY=$3
export TOKEN_CONTROLLER_DEPLOYER_KEY=$3
export ATTESTER_ADDRESS=$7
export USDC_CONTRACT_ADDRESS=$4
export TOKEN_CONTROLLER_ADDRESS=$2
export BURN_LIMIT_PER_MESSAGE=1000
export REMOTE_USDC_CONTRACT_ADDRESS=$5
export DOMAIN=1
export REMOTE_DOMAIN=2
export MESSAGE_TRANSMITTER_PAUSER_ADDRESS=$2
export TOKEN_MINTER_PAUSER_ADDRESS=$2
export MESSAGE_TRANSMITTER_RESCUER_ADDRESS=$2
export TOKEN_MESSENGER_RESCUER_ADDRESS=$2
export TOKEN_MINTER_RESCUER_ADDRESS=$2
export REMOTE_TOKEN_MESSENGER_ADDRESS=$6


make simulate RPC_URL=http://localhost:8545 SENDER=$2

Dockerfile
LICENSE
Makefile
NOTICES
README.md
anvil
broadcast
cache
docs
foundry.toml
lib
out
package.json
pictures
remappings.txt
requirements.txt
scripts
slither.config.json
src
test
yarn.lock
0x892BB2e4F6b14a2B5b82Ba8d33E5925D42D4431F
forge script scripts/deploy.s.sol:DeployScript --rpc-url http://localhost:8545 --sender 0x892BB2e4F6b14a2B5b82Ba8d33E5925D42D4431F
Nothing to compile
Script ran successfully.

EIP-3855 is not supported in one or more of the RPCs used.
Unsupported Chain IDs: 31337.
Contracts deployed with a Solidity version equal or higher than 0.8.20 might not work properly.
For more information, please see https://eips.ethereum.org/EIPS/eip-3855



## Setting up (1) EVMs.




Chain 31337

Estimated gas price: 3.00000444 gwei

Estimated total gas used for script: 12236923

Estimated amount required: 0.03671082333193812 ETH


SIMULATION COMPLETE. To broadcast these transactions, add --broadcast and wallet configuration(s) to the previous command. See forge script --help for more.

Transactions saved to: /home/dsmith/code/evm-cctp-contracts/broadcast/deploy.s.sol/31337/dry-run/run-latest.json

Sensitive values saved to: /home/dsmith/code/evm-cctp-contracts/cache/deploy.s.sol/31337/dry-run/run-latest.json



In [17]:
%%bash -s "$cctp_home" "$domain_1_acct" "$domain_1_key" "$DOMAIN1_FIDDY_ADDRESS" "$DOMAIN2_FIDDY_ADDRESS" "$domain2TokenMessengerAddress" "$attestor"

cd $1
ls $1
echo $2
export MESSAGE_TRANSMITTER_DEPLOYER_KEY=$3
export TOKEN_MESSENGER_DEPLOYER_KEY=$3
export TOKEN_MINTER_DEPLOYER_KEY=$3
export TOKEN_CONTROLLER_DEPLOYER_KEY=$3
export ATTESTER_ADDRESS=$7
export USDC_CONTRACT_ADDRESS=$4
export TOKEN_CONTROLLER_ADDRESS=$2
export BURN_LIMIT_PER_MESSAGE=1000
export REMOTE_USDC_CONTRACT_ADDRESS=$5
export DOMAIN=1
export REMOTE_DOMAIN=2
export MESSAGE_TRANSMITTER_PAUSER_ADDRESS=$2
export TOKEN_MINTER_PAUSER_ADDRESS=$2
export MESSAGE_TRANSMITTER_RESCUER_ADDRESS=$2
export TOKEN_MESSENGER_RESCUER_ADDRESS=$2
export TOKEN_MINTER_RESCUER_ADDRESS=$2
export REMOTE_TOKEN_MESSENGER_ADDRESS=$6


make deploy RPC_URL=http://localhost:8545 SENDER=$2


Dockerfile
LICENSE
Makefile
NOTICES
README.md
anvil
broadcast
cache
docs
foundry.toml
lib
out
package.json
pictures
remappings.txt
requirements.txt
scripts
slither.config.json
src
test
yarn.lock
0x892BB2e4F6b14a2B5b82Ba8d33E5925D42D4431F
forge script --slow scripts/deploy.s.sol:DeployScript --rpc-url http://localhost:8545 --sender 0x892BB2e4F6b14a2B5b82Ba8d33E5925D42D4431F --broadcast
Nothing to compile
Script ran successfully.

EIP-3855 is not supported in one or more of the RPCs used.
Unsupported Chain IDs: 31337.
Contracts deployed with a Solidity version equal or higher than 0.8.20 might not work properly.
For more information, please see https://eips.ethereum.org/EIPS/eip-3855



## Setting up (1) EVMs.




Chain 31337

Estimated gas price: 3.00000444 gwei

Estimated total gas used for script: 12236923

Estimated amount required: 0.03671082333193812 ETH


###
Finding wallets for all the necessary addresses...
##
Sending transactions [0 - 13].

##### anvil-hardhat
✅  [Success]Hash: 0x33e0614442bb31ec5554ef8ccc823df14e2249ffc89a8857e7e7891bb9ba3b93
Contract Address: 0xeAf6b136BCCD73bD98000A9000F78AC2d40c0c30
Block: 104
Paid: 0.000342231223819074 ETH (114077 gas * 3.000001962 gwei)


##### anvil-hardhat
✅  [Success]Hash: 0x37c8dfadfdfce17c86c9c8aa922b9f69e8bb43a1c93fdc4116f2c6d3ad4bc018
Contract Address: 0x4b8e6646Fbf67e6aBDe1352B7fE61BCB5252EB19
Block: 105
Paid: 0.011983215866378757 ETH (3994403 gas * 3.000001719 gwei)


##### anvil-hardhat
✅  [Success]Hash: 0x5a21a583b0734d89847fc8459faff8eac3de7e56cf7ec9ea5fde01e5827b4c3b
Block: 106
Paid: 0.00014163007374202 ETH (47210 gas * 3.000001562 gwei)


##### anvil-hardhat
✅  [Success]Hash: 0x189989aeb69928e86f03d2b6782232ddab949824efc51b3957a6a

In [18]:
#export CCTPCTX=$HOME/code/evm-cctp-contracts/broadcast/deploy.s.sol/31337/run-latest.json
cctp_ctx = home[0] + '/code/evm-cctp-contracts/broadcast/deploy.s.sol/31337/run-latest.json'
print(cctp_ctx)

/home/dsmith/code/evm-cctp-contracts/broadcast/deploy.s.sol/31337/run-latest.json


In [19]:
%%bash -s "$exec_home" "$cctp_ctx" --out CCTP_AGGREGATE
cd $1
export CCTPCTX=$2
ls
node script/cctp-details.js

In [20]:
CCTP_AGGREGATE = CCTP_AGGREGATE.strip()
parts = CCTP_AGGREGATE.split(':')
print(parts)

['0x4b8e6646Fbf67e6aBDe1352B7fE61BCB5252EB19', '0x7649eAA2E7229ada48C34E58583CDd2fceA0BFEa', '0xB048dFcCB459121b786de46DcAA81361495F9eE2']


In [21]:
domain1MessageTransmitterAddr = parts[0]
domain1TokenMessengerAddr = parts[1]
domain1TokenMinterAddr = parts[2]

In [22]:
print(f'domain1MessageTransmitterAddr {domain1MessageTransmitterAddr}')
print(f'domain1TokenMessengerAddr     {domain1TokenMessengerAddr}')
print(f'domain1TokenMinterAddr        {domain1TokenMinterAddr}')

domain1MessageTransmitterAddr 0x4b8e6646Fbf67e6aBDe1352B7fE61BCB5252EB19
domain1TokenMessengerAddr     0x7649eAA2E7229ada48C34E58583CDd2fceA0BFEa
domain1TokenMinterAddr        0xB048dFcCB459121b786de46DcAA81361495F9eE2


## CCTP Setup - Domain 2

In [23]:
%%bash -s "$cctp_home" "$domain_2_acct" "$domain_2_key" "$DOMAIN2_FIDDY_ADDRESS" "$DOMAIN1_FIDDY_ADDRESS" "$domain1TokenMessengerAddr" "$attestor"

cd $1
ls $1
echo $2
export MESSAGE_TRANSMITTER_DEPLOYER_KEY=$3
export TOKEN_MESSENGER_DEPLOYER_KEY=$3
export TOKEN_MINTER_DEPLOYER_KEY=$3
export TOKEN_CONTROLLER_DEPLOYER_KEY=$3
export ATTESTER_ADDRESS=$7
export USDC_CONTRACT_ADDRESS=$4
export TOKEN_CONTROLLER_ADDRESS=$2
export BURN_LIMIT_PER_MESSAGE=1000
export REMOTE_USDC_CONTRACT_ADDRESS=$5
export DOMAIN=2
export REMOTE_DOMAIN=1
export MESSAGE_TRANSMITTER_PAUSER_ADDRESS=$2
export TOKEN_MINTER_PAUSER_ADDRESS=$2
export MESSAGE_TRANSMITTER_RESCUER_ADDRESS=$2
export TOKEN_MESSENGER_RESCUER_ADDRESS=$2
export TOKEN_MINTER_RESCUER_ADDRESS=$2
export REMOTE_TOKEN_MESSENGER_ADDRESS=$6


make simulate RPC_URL=http://localhost:8545 SENDER=$2

Dockerfile
LICENSE
Makefile
NOTICES
README.md
anvil
broadcast
cache
docs
foundry.toml
lib
out
package.json
pictures
remappings.txt
requirements.txt
scripts
slither.config.json
src
test
yarn.lock
0x73dA1eD554De26C467d97ADE090af6d52851745E
forge script scripts/deploy.s.sol:DeployScript --rpc-url http://localhost:8545 --sender 0x73dA1eD554De26C467d97ADE090af6d52851745E
Nothing to compile
Script ran successfully.

EIP-3855 is not supported in one or more of the RPCs used.
Unsupported Chain IDs: 31337.
Contracts deployed with a Solidity version equal or higher than 0.8.20 might not work properly.
For more information, please see https://eips.ethereum.org/EIPS/eip-3855



## Setting up (1) EVMs.




Chain 31337

Estimated gas price: 3.000000762 gwei

Estimated total gas used for script: 12237203

Estimated amount required: 0.036711618324748686 ETH


SIMULATION COMPLETE. To broadcast these transactions, add --broadcast and wallet configuration(s) to the previous command. See forge script --help for more.

Transactions saved to: /home/dsmith/code/evm-cctp-contracts/broadcast/deploy.s.sol/31337/dry-run/run-latest.json

Sensitive values saved to: /home/dsmith/code/evm-cctp-contracts/cache/deploy.s.sol/31337/dry-run/run-latest.json



In [24]:
%%bash -s "$cctp_home" "$domain_2_acct" "$domain_2_key" "$DOMAIN2_FIDDY_ADDRESS" "$DOMAIN1_FIDDY_ADDRESS" "$domain1TokenMessengerAddr" "$attestor"

cd $1
ls $1
echo $2
export MESSAGE_TRANSMITTER_DEPLOYER_KEY=$3
export TOKEN_MESSENGER_DEPLOYER_KEY=$3
export TOKEN_MINTER_DEPLOYER_KEY=$3
export TOKEN_CONTROLLER_DEPLOYER_KEY=$3
export ATTESTER_ADDRESS=$7
export USDC_CONTRACT_ADDRESS=$4
export TOKEN_CONTROLLER_ADDRESS=$2
export BURN_LIMIT_PER_MESSAGE=1000
export REMOTE_USDC_CONTRACT_ADDRESS=$5
export DOMAIN=2
export REMOTE_DOMAIN=1
export MESSAGE_TRANSMITTER_PAUSER_ADDRESS=$2
export TOKEN_MINTER_PAUSER_ADDRESS=$2
export MESSAGE_TRANSMITTER_RESCUER_ADDRESS=$2
export TOKEN_MESSENGER_RESCUER_ADDRESS=$2
export TOKEN_MINTER_RESCUER_ADDRESS=$2
export REMOTE_TOKEN_MESSENGER_ADDRESS=$6


make deploy RPC_URL=http://localhost:8545 SENDER=$2

Dockerfile
LICENSE
Makefile
NOTICES
README.md
anvil
broadcast
cache
docs
foundry.toml
lib
out
package.json
pictures
remappings.txt
requirements.txt
scripts
slither.config.json
src
test
yarn.lock
0x73dA1eD554De26C467d97ADE090af6d52851745E
forge script --slow scripts/deploy.s.sol:DeployScript --rpc-url http://localhost:8545 --sender 0x73dA1eD554De26C467d97ADE090af6d52851745E --broadcast
Nothing to compile
Script ran successfully.

EIP-3855 is not supported in one or more of the RPCs used.
Unsupported Chain IDs: 31337.
Contracts deployed with a Solidity version equal or higher than 0.8.20 might not work properly.
For more information, please see https://eips.ethereum.org/EIPS/eip-3855



## Setting up (1) EVMs.




Chain 31337

Estimated gas price: 3.000000762 gwei

Estimated total gas used for script: 12237203

Estimated amount required: 0.036711618324748686 ETH


###
Finding wallets for all the necessary addresses...
##
Sending transactions [0 - 13].

##### anvil-hardhat
✅  [Success]Hash: 0x9227c0140e6ee4d01b0cf493864e818ba3ba49d5f7088c7bc3e361988b7ff1a7
Contract Address: 0x71ED4A559C87F6D650A00744B5cCD9687153029a
Block: 118
Paid: 0.000342231038101718 ETH (114077 gas * 3.000000334 gwei)


##### anvil-hardhat
✅  [Success]Hash: 0x37a2c70adf27dc8dfd066c46ced4c84abfb3e9e6d71205cc2460764228a3f9b2
Contract Address: 0xe7793E36AAf773A408d28cf5873c671b684F12b3
Block: 119
Paid: 0.011983570170395239 ETH (3994523 gas * 3.000000293 gwei)


##### anvil-hardhat
✅  [Success]Hash: 0x46decc7591662cd633d84e78f7a739cdddf969c1c95dc1f57a7fbc45332ad147
Block: 120
Paid: 0.00014163001260507 ETH (47210 gas * 3.000000267 gwei)


##### anvil-hardhat
✅  [Success]Hash: 0x11e68b12bd05d42e637bebd2cbaf86023507b3194da4963e3cb

In [25]:
%%bash -s "$exec_home" "$cctp_ctx" --out CCTP_AGGREGATE
cd $1
export CCTPCTX=$2
ls
node script/cctp-details.js

In [26]:
CCTP_AGGREGATE = CCTP_AGGREGATE.strip()
parts = CCTP_AGGREGATE.split(':')
print(parts)

['0xe7793E36AAf773A408d28cf5873c671b684F12b3', '0x46Cd74668952F7c935D585BdfbB4EC524A9d4Af7', '0xF1ABDd310E026aa7D797298Ea4D4c32447f7049B']


In [27]:
domain2MessageTransmitterAddr = parts[0]
domain2TokenMessengerAddr = parts[1]
domain2TokenMinterAddr = parts[2]

In [28]:
print(f'domain2MessageTransmitterAddr { domain2MessageTransmitterAddr }')
print(f'domain2TokenMessengerAddr     { domain2TokenMessengerAddr }')
print(f'domain2TokenMinterAddr        { domain2TokenMinterAddr }')

domain2MessageTransmitterAddr 0xe7793E36AAf773A408d28cf5873c671b684F12b3
domain2TokenMessengerAddr     0x46Cd74668952F7c935D585BdfbB4EC524A9d4Af7
domain2TokenMinterAddr        0xF1ABDd310E026aa7D797298Ea4D4c32447f7049B


## Fix Token Messenger if Needed

***Note - looks like the first contract deployed is Message, which means it won't match the precompute - need to adjust logic***

In [29]:
# Did the precompute match the deployment address?
print(domain2TokenMessengerAddr)
print(domain2TokenMessengerAddress)
domain2TokenMessengerAddr == domain2TokenMessengerAddress

0x46Cd74668952F7c935D585BdfbB4EC524A9d4Af7
71ed4a559c87f6d650a00744b5ccd9687153029a


False

In [30]:
domain2TokenMessAddrBytes32 = bytes.fromhex(domain2TokenMessengerAddr[2:])
domain2TokenMessAddrBytes32 =  b'\0'*12 + domain2TokenMessAddrBytes32
print(domain2TokenMessAddrBytes32.hex())

00000000000000000000000046cd74668952f7c935d585bdfbb4ec524a9d4af7


In [31]:
ethTokenMessengerAbiFile = home[0] + '/code/evm-cctp-contracts/out/TokenMessenger.sol/TokenMessenger.json'
print(ethTokenMessengerAbiFile)

/home/dsmith/code/evm-cctp-contracts/out/TokenMessenger.sol/TokenMessenger.json


In [32]:
tokenMessengerAbiFile = home[0] + '/code/evm-cctp-contracts/out/TokenMessenger.sol/TokenMessenger.json'
print(tokenMessengerAbiFile)

/home/dsmith/code/evm-cctp-contracts/out/TokenMessenger.sol/TokenMessenger.json


In [33]:
import json

with open(tokenMessengerAbiFile) as f:
    tm_json = json.load(f)

tokenMessengerAbi = tm_json['abi']

In [34]:
print(tokenMessengerAbi)

[{'inputs': [{'internalType': 'address', 'name': '_messageTransmitter', 'type': 'address'}, {'internalType': 'uint32', 'name': '_messageBodyVersion', 'type': 'uint32'}], 'stateMutability': 'nonpayable', 'type': 'constructor'}, {'anonymous': False, 'inputs': [{'indexed': True, 'internalType': 'uint64', 'name': 'nonce', 'type': 'uint64'}, {'indexed': True, 'internalType': 'address', 'name': 'burnToken', 'type': 'address'}, {'indexed': False, 'internalType': 'uint256', 'name': 'amount', 'type': 'uint256'}, {'indexed': True, 'internalType': 'address', 'name': 'depositor', 'type': 'address'}, {'indexed': False, 'internalType': 'bytes32', 'name': 'mintRecipient', 'type': 'bytes32'}, {'indexed': False, 'internalType': 'uint32', 'name': 'destinationDomain', 'type': 'uint32'}, {'indexed': False, 'internalType': 'bytes32', 'name': 'destinationTokenMessenger', 'type': 'bytes32'}, {'indexed': False, 'internalType': 'bytes32', 'name': 'destinationCaller', 'type': 'bytes32'}], 'name': 'DepositForBur

In [35]:
from web3 import Web3
w3 = Web3(Web3.HTTPProvider('http://127.0.0.1:8545'))
domain1TokenMessenger = w3.eth.contract(address=domain1TokenMessengerAddr, abi=tokenMessengerAbi)

In [36]:
domain1TokenMessenger.functions.remoteTokenMessengers(2).call().hex()

'00000000000000000000000071ed4a559c87f6d650a00744b5ccd9687153029a'

In [37]:
tx = domain1TokenMessenger.functions.removeRemoteTokenMessenger(2).build_transaction({
    "from":domain_1_acct,
    "nonce": w3.eth.get_transaction_count(domain_1_acct),
})

In [38]:
signed_tx = w3.eth.account.sign_transaction(tx, domain_1_key)
tx_hash = w3.eth.send_raw_transaction(signed_tx.rawTransaction)
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
print(tx_receipt)
print(f"Tx successful with hash: { tx_receipt.transactionHash.hex() }")

AttributeDict({'transactionHash': HexBytes('0x9d60b998628a0010568a11087a8bf6d2f15f6208f4aac6ed04d633c29bf0da5d'), 'transactionIndex': 0, 'blockHash': HexBytes('0xd71cb2b702a0f83a5159181e633515a5cdd3acd3bc1d6d5792756af09b672f05'), 'blockNumber': 132, 'from': '0x892BB2e4F6b14a2B5b82Ba8d33E5925D42D4431F', 'to': '0x7649eAA2E7229ada48C34E58583CDd2fceA0BFEa', 'cumulativeGasUsed': 25518, 'gasUsed': 25518, 'contractAddress': None, 'logs': [AttributeDict({'address': '0x7649eAA2E7229ada48C34E58583CDd2fceA0BFEa', 'topics': [HexBytes('0x3dcea012093dbca2bb8ed7fd2b2ff90305ab70bddda8bbb94d4152735a98f0b1')], 'data': '0x000000000000000000000000000000000000000000000000000000000000000200000000000000000000000071ed4a559c87f6d650a00744b5ccd9687153029a', 'blockHash': HexBytes('0xd71cb2b702a0f83a5159181e633515a5cdd3acd3bc1d6d5792756af09b672f05'), 'blockNumber': 132, 'transactionHash': HexBytes('0x9d60b998628a0010568a11087a8bf6d2f15f6208f4aac6ed04d633c29bf0da5d'), 'transactionIndex': 0, 'logIndex': 0, 'transac

In [39]:
tx = domain1TokenMessenger.functions.addRemoteTokenMessenger(2, domain2TokenMessAddrBytes32).build_transaction({
    "from":ACCT0,
    "nonce": w3.eth.get_transaction_count(domain_1_acct),
})

In [40]:
signed_tx = w3.eth.account.sign_transaction(tx, domain_1_key)
tx_hash = w3.eth.send_raw_transaction(signed_tx.rawTransaction)
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
print(tx_receipt)
print(f"Tx successful with hash: { tx_receipt.transactionHash.hex() }")

AttributeDict({'transactionHash': HexBytes('0xa904cf21b6ecd213e22cd93e6c1352566341fcd922ec9f045008296ff3aecd27'), 'transactionIndex': 0, 'blockHash': HexBytes('0xb2eec1d52b2bf7fcee5ee0a2ad36b3a637558d07a6562d7dd35bc001f8e51c74'), 'blockNumber': 133, 'from': '0x892BB2e4F6b14a2B5b82Ba8d33E5925D42D4431F', 'to': '0x7649eAA2E7229ada48C34E58583CDd2fceA0BFEa', 'cumulativeGasUsed': 47688, 'gasUsed': 47688, 'contractAddress': None, 'logs': [AttributeDict({'address': '0x7649eAA2E7229ada48C34E58583CDd2fceA0BFEa', 'topics': [HexBytes('0x4bba2b08298cf59661b4895e384cc2ac3962ce2d71f1b7c11bca52e1169f9599')], 'data': '0x000000000000000000000000000000000000000000000000000000000000000200000000000000000000000046cd74668952f7c935d585bdfbb4ec524a9d4af7', 'blockHash': HexBytes('0xb2eec1d52b2bf7fcee5ee0a2ad36b3a637558d07a6562d7dd35bc001f8e51c74'), 'blockNumber': 133, 'transactionHash': HexBytes('0xa904cf21b6ecd213e22cd93e6c1352566341fcd922ec9f045008296ff3aecd27'), 'transactionIndex': 0, 'logIndex': 0, 'transac

In [41]:
domain1TokenMessenger.functions.remoteTokenMessengers(2).call().hex()

'00000000000000000000000046cd74668952f7c935d585bdfbb4ec524a9d4af7'

## Working with Fiddy and CCTP on Domain 1

In [42]:
fiddyAbiFile = home[0] + '/code/cctp-components/out/Token.sol/FiddyCent.json'

In [43]:
import json

with open(fiddyAbiFile) as f:
    fiddy_json = json.load(f)

fiddyAbi = fiddy_json['abi']


In [44]:
domain1Fiddy = w3.eth.contract(address=DOMAIN1_FIDDY_ADDRESS, abi=fiddyAbi)

In [45]:
# Initial balance
domain1Fiddy.functions.balanceOf(ACCT1).call()

0

In [46]:
# Transfer some fiddy coin to acct1
tx = domain1Fiddy.functions.transfer(ACCT1, 100).build_transaction(
    {
        "from":ACCT0,
        "nonce": w3.eth.get_transaction_count(ACCT0),
    }
)

In [47]:
signed_tx = w3.eth.account.sign_transaction(tx, ACCT0KEY)

In [48]:
tx_hash = w3.eth.send_raw_transaction(signed_tx.rawTransaction)
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)

print(f"Tx successful with hash: { tx_receipt.transactionHash.hex() }")

Tx successful with hash: 0x2eeda0c047930d27b989776f3259c7880131a8101188015749b9b7f9b49ca713


In [49]:
domain1Fiddy.functions.balanceOf(ACCT1).call()

100

In [50]:
# Check the allowance for ACCT1 on the messenger contract
domain1Fiddy.functions.allowance(ACCT1, domain1TokenMessengerAddr).call()

0

In [51]:
# Set an allowance
tx = domain1Fiddy.functions.approve(domain1TokenMessengerAddr, 100).build_transaction(
    {
        "from":ACCT1,
        "nonce": w3.eth.get_transaction_count(ACCT1),
    }
)

In [52]:
signed_tx = w3.eth.account.sign_transaction(tx, ACCT1KEY)
tx_hash = w3.eth.send_raw_transaction(signed_tx.rawTransaction)
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)

print(f"Tx successful with hash: { tx_receipt.transactionHash.hex() }")
print(f"New allowance: { domain1Fiddy.functions.allowance(ACCT1, domain1TokenMessengerAddr).call() }")

Tx successful with hash: 0xb5e86e684327fb1d2e082d53c3cb43b01238e576e9b9d8e13cd3f2edaaf3abb3
New allowance: 100


## Deposit for Burn - Domain 1 Side

In [53]:
domain1TokenMessenger.functions.remoteTokenMessengers(2).call().hex()

'00000000000000000000000046cd74668952f7c935d585bdfbb4ec524a9d4af7'

In [54]:
# Deposit some coinage for burn - args are amount, destination domain, mint recipient, contract that burns the deposit
tx = domain1TokenMessenger.functions.depositForBurn(1, 2, domain_2_mint_recipient, DOMAIN1_FIDDY_ADDRESS).build_transaction(
     {
        "from":ACCT1,
        "nonce": w3.eth.get_transaction_count(ACCT1),
    }
)

In [55]:
signed_tx = w3.eth.account.sign_transaction(tx, ACCT1KEY)
tx_hash = w3.eth.send_raw_transaction(signed_tx.rawTransaction)
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
print(tx_receipt)
print(f"Tx successful with hash: { tx_receipt.transactionHash.hex() }")

AttributeDict({'transactionHash': HexBytes('0x82b33b9e8138a09656f30bc3e108656e85e1fc04e97acfbab40a20f6ec290164'), 'transactionIndex': 0, 'blockHash': HexBytes('0x79b08faf6ca3748d99f525375af6f2bbd4cd1663266fca1f1fc7caea180e1c99'), 'blockNumber': 136, 'from': '0x9949f7e672a568bB3EBEB777D5e8D1c1107e96E5', 'to': '0x7649eAA2E7229ada48C34E58583CDd2fceA0BFEa', 'cumulativeGasUsed': 110288, 'gasUsed': 110288, 'contractAddress': None, 'logs': [AttributeDict({'address': '0x13B2DC341d5Ce2F9FF32c7e8aeFAe96ff2A35998', 'topics': [HexBytes('0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef'), HexBytes('0x0000000000000000000000009949f7e672a568bb3ebeb777d5e8d1c1107e96e5'), HexBytes('0x000000000000000000000000b048dfccb459121b786de46dcaa81361495f9ee2')], 'data': '0x0000000000000000000000000000000000000000000000000000000000000001', 'blockHash': HexBytes('0x79b08faf6ca3748d99f525375af6f2bbd4cd1663266fca1f1fc7caea180e1c99'), 'blockNumber': 136, 'transactionHash': HexBytes('0x82b33b9e8138a096

In [56]:
domain1Fiddy.functions.balanceOf(ACCT1).call()

99

In [57]:
domain1Fiddy.functions.totalSupply().call()

999999999999999999999999999

## Message Transmitter Events

In [58]:
msgTransmitterAbiFile = home[0] + '/code/evm-cctp-contracts/out/MessageTransmitter.sol/MessageTransmitter.json'
print(msgTransmitterAbiFile)

/home/dsmith/code/evm-cctp-contracts/out/MessageTransmitter.sol/MessageTransmitter.json


In [59]:
with open(msgTransmitterAbiFile) as f:
    mt_json = json.load(f)

msgTransmitterAbi = mt_json['abi']

In [60]:
print(msgTransmitterAbi)

[{'inputs': [{'internalType': 'uint32', 'name': '_localDomain', 'type': 'uint32'}, {'internalType': 'address', 'name': '_attester', 'type': 'address'}, {'internalType': 'uint32', 'name': '_maxMessageBodySize', 'type': 'uint32'}, {'internalType': 'uint32', 'name': '_version', 'type': 'uint32'}], 'stateMutability': 'nonpayable', 'type': 'constructor'}, {'anonymous': False, 'inputs': [{'indexed': True, 'internalType': 'address', 'name': 'attester', 'type': 'address'}], 'name': 'AttesterDisabled', 'type': 'event'}, {'anonymous': False, 'inputs': [{'indexed': True, 'internalType': 'address', 'name': 'attester', 'type': 'address'}], 'name': 'AttesterEnabled', 'type': 'event'}, {'anonymous': False, 'inputs': [{'indexed': True, 'internalType': 'address', 'name': 'previousAttesterManager', 'type': 'address'}, {'indexed': True, 'internalType': 'address', 'name': 'newAttesterManager', 'type': 'address'}], 'name': 'AttesterManagerUpdated', 'type': 'event'}, {'anonymous': False, 'inputs': [{'indexe

In [61]:
domain1MessageTransmitter = w3.eth.contract(address=domain1MessageTransmitterAddr, abi=msgTransmitterAbi)

In [62]:
messageSentEvents = domain1MessageTransmitter.events.MessageSent.getLogs(fromBlock=1)
print(messageSentEvents)

(AttributeDict({'args': AttributeDict({'message': b'\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00vI\xea\xa2\xe7"\x9a\xdaH\xc3NXX<\xdd/\xce\xa0\xbf\xea\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00F\xcdtf\x89R\xf7\xc95\xd5\x85\xbd\xfb\xb4\xecRJ\x9dJ\xf7\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x13\xb2\xdc4\x1d\\\xe2\xf9\xff2\xc7\xe8\xae\xfa\xe9o\xf2\xa3Y\x98\xb4\xc3\xd7\x9c\xdc\x0e\xb7\xa8Wj\x8b\xf2$\xbb\xc6\xbe\xc7\x90\xc3 \x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x99I\xf7\xe6r\xa5h\xbb>\xbe\xb7w\xd5\xe8\xd1\xc1\x10~\x96\xe5'}), 'event': 'MessageSent', 'logIndex': 2, 'transa

In [63]:
type(messageSentEvents)

tuple

In [64]:
len(messageSentEvents)

1

In [65]:
lastMessageSentEvent = messageSentEvents[len(messageSentEvents) - 1]

In [66]:
lastMessageSentEvent

AttributeDict({'args': AttributeDict({'message': b'\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00vI\xea\xa2\xe7"\x9a\xdaH\xc3NXX<\xdd/\xce\xa0\xbf\xea\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00F\xcdtf\x89R\xf7\xc95\xd5\x85\xbd\xfb\xb4\xecRJ\x9dJ\xf7\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x13\xb2\xdc4\x1d\\\xe2\xf9\xff2\xc7\xe8\xae\xfa\xe9o\xf2\xa3Y\x98\xb4\xc3\xd7\x9c\xdc\x0e\xb7\xa8Wj\x8b\xf2$\xbb\xc6\xbe\xc7\x90\xc3 \x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x99I\xf7\xe6r\xa5h\xbb>\xbe\xb7w\xd5\xe8\xd1\xc1\x10~\x96\xe5'}),
 'event': 'MessageSent',
 'logIndex': 2,
 'tran

In [67]:
messageSentBytes = lastMessageSentEvent.args.message
print(messageSentBytes.hex())

00000000000000010000000200000000000000000000000000000000000000007649eaa2e7229ada48c34e58583cdd2fcea0bfea00000000000000000000000046cd74668952f7c935d585bdfbb4ec524a9d4af700000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000013b2dc341d5ce2f9ff32c7e8aefae96ff2a35998b4c3d79cdc0eb7a8576a8bf224bbc6bec790c32000000000000000000000000000000000000000000000000000000000000000000000000000000000000000010000000000000000000000009949f7e672a568bb3ebeb777d5e8d1c1107e96e5


## Attestation



In [68]:
mbHash = Web3.keccak(messageSentBytes)
signedHash = w3.eth.account.signHash(mbHash, private_key=attestor_key)

## Redeem Using Attested Message

In [83]:
# To redeem on polygon, we call ReceiveMessage on message transmitter
domain2MessageTransmitter = w3.eth.contract(address=domain2MessageTransmitterAddr, abi=msgTransmitterAbi)

In [84]:
domain2MessageTransmitterAddr

'0xe7793E36AAf773A408d28cf5873c671b684F12b3'

In [85]:
domain1MessageTransmitterAddr

'0x4b8e6646Fbf67e6aBDe1352B7fE61BCB5252EB19'

In [86]:
domain1MessageTransmitter.functions.localDomain().call()

1

In [87]:
domain2MessageTransmitter.functions.localDomain().call()

2

In [88]:
domain2MessageTransmitter.functions.nextAvailableNonce().call()

0

In [89]:
domain2MessageTransmitter.functions.getNumEnabledAttesters().call()

1

In [90]:
domain2MessageTransmitter.functions.getEnabledAttester(0).call()

'0x549381D65fe61046911d11743D5c0941Ed704640'

In [91]:
domain2MessageTransmitter.functions.getEnabledAttester(0).call() == attestor

True

In [92]:
messageSentBytes.hex()

'00000000000000010000000200000000000000000000000000000000000000007649eaa2e7229ada48c34e58583cdd2fcea0bfea00000000000000000000000046cd74668952f7c935d585bdfbb4ec524a9d4af700000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000013b2dc341d5ce2f9ff32c7e8aefae96ff2a35998b4c3d79cdc0eb7a8576a8bf224bbc6bec790c32000000000000000000000000000000000000000000000000000000000000000000000000000000000000000010000000000000000000000009949f7e672a568bb3ebeb777d5e8d1c1107e96e5'

In [93]:
domain2TokenMessenger = w3.eth.contract(address=domain2TokenMessengerAddr, abi=tokenMessengerAbi)

In [94]:
domain2TokenMessenger.functions.localMinter().call()

'0xF1ABDd310E026aa7D797298Ea4D4c32447f7049B'

In [97]:
domain2MessageTransmitter.functions.isEnabledAttester(attestor).call()

True

In [ ]:
tx = domain2MessageTransmitter.functions.receiveMessage(messageSentBytes,signedHash.signature).build_transaction({
    "from":domain_2_acct,
    "nonce": w3.eth.get_transaction_count(domain_2_acct),
})

In [ ]:
pgMessageTransmitter.functions.